In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install scikit-plot

In [ ]:
from plotly.offline import init_notebook_mode, iplot, plot
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tqdm import tqdm
import pandas as pd
import numpy as np
import pickle
import re
from sklearn.preprocessing import Normalizer
from scipy.io import arff
init_notebook_mode(connected=True)
from keras.layers import Dense, Dropout, Activation, Embedding
from keras.utils.np_utils import to_categorical
from keras.layers import LSTM
from keras.layers import Bidirectional
from keras.models import Sequential
import scikitplot as skplt
import h5py
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error,roc_auc_score,cohen_kappa_score)

In [ ]:
 #load training dataset
dataTrain = arff.loadarff('/content/drive/MyDrive/KDDTrain20MultiClass.arff')
dataset_train = pd.DataFrame(dataTrain[0])
dataTest = arff.loadarff('/content/drive/My Drive/KDDTest21MultiClass.arff')
dataset_test= pd.DataFrame(dataTrain[0])

X_train = dataset_train.iloc[:, :-2].values
Y_train = dataset_train.iloc[:, 41].values

X_test = dataset_test.iloc[:, :-2].values
Y_test = dataset_test.iloc[:, 41].values 

In [ ]:
classes=[0,1,2,3,4,5]
n_classes = 6

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

X_train.shape

X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

x_train = np.array(X_train)
x_test = np.array(X_test)
y_train1 = np.array(Y_train)
y_test1 = np.array(Y_test)

y_train= to_categorical(y_train1)
y_test= to_categorical(y_test1)

In [ ]:
model = Sequential()
model.add(Bidirectional(LSTM(80, input_dim=40, return_sequences=True)))
model.add(Dropout(0.1))
model.add(Bidirectional(LSTM(40)))
model.add(Dropout(0.1))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(6, activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
#model.summary()

model.fit(X_train, y_train, epochs=15, batch_size=64, verbose=1)

# evaluate the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
# save model and architecture to single file
model.save("model.h5")
print("Saved model to disk")

Epoch 1/15
394/394 [==============================] - 3s 7ms/step - loss: 0.2323 - accuracy: 0.9495
Epoch 2/15
394/394 [==============================] - 2s 6ms/step - loss: 0.0512 - accuracy: 0.9824
Epoch 3/15
394/394 [==============================] - 3s 6ms/step - loss: 0.0403 - accuracy: 0.9862
Epoch 4/15
394/394 [==============================] - 2s 6ms/step - loss: 0.0336 - accuracy: 0.9886
Epoch 5/15
394/394 [==============================] - 3s 6ms/step - loss: 0.0318 - accuracy: 0.9889
Epoch 6/15
394/394 [==============================] - 2s 6ms/step - loss: 0.0293 - accuracy: 0.9896
Epoch 7/15
394/394 [==============================] - 3s 7ms/step - loss: 0.0276 - accuracy: 0.9903
Epoch 8/15
394/394 [==============================] - 2s 6ms/step - loss: 0.0258 - accuracy: 0.9909
Epoch 9/15
394/394 [==============================] - 2s 6ms/step - loss: 0.0252 - accuracy: 0.9914
Epoch 10/15
394/394 [==============================] - 3s 6ms/step - loss: 0.0236 - accuracy: 0.9923

In [ ]:
target_names = ['DoS', 'U2R', 'R2L', 'Probe','Normal','Unknown']

In [ ]:
### train_accuracy
import sklearn.metrics as metrics
expected = y_train
predicted = model.predict(X_train)
data = classification_report(expected,np.round(predicted), target_names=target_names)
print(data)

print("Accuracy_Train:",metrics.accuracy_score(expected, np.round(predicted)))
print("Precision_Train:",metrics.precision_score(expected, np.round(predicted), average='weighted'))
print("Recall_Train:",metrics.recall_score(expected, np.round(predicted), average='weighted'))
print("f1-score_Train:",metrics.f1_score(expected, np.round(predicted), average='weighted'))

              precision    recall  f1-score   support

         DoS       0.00      0.00      0.00         0
         U2R       1.00      1.00      1.00      9234
         R2L       1.00      0.55      0.71        11
       Probe       0.82      0.84      0.83       209
      Normal       0.98      0.98      0.98      2289
     Unknown       0.99      0.99      0.99     13449

   micro avg       0.99      0.99      0.99     25192
   macro avg       0.80      0.73      0.75     25192
weighted avg       0.99      0.99      0.99     25192
 samples avg       0.99      0.99      0.99     25192

Accuracy_Train: 0.9927357891394093
Precision_Train: 0.9928965462384829
Recall_Train: 0.9927357891394093
f1-score_Train: 0.9927852714648865


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this 

In [ ]:
### test_accuracy
import sklearn.metrics as metrics
target_names = ['DoS', 'U2R', 'R2L', 'Probe', 'Normal', 'Unknown']
expected = y_test
predicted = model.predict(X_test)

print("Accuracy:",metrics.accuracy_score(expected, np.round(predicted)))
print("Precision:",metrics.precision_score(expected, np.round(predicted), average='weighted'))
print("Recall:",metrics.recall_score(expected, np.round(predicted), average='weighted'))
print("f1-score:",metrics.f1_score(expected, np.round(predicted), average='weighted'))
expected_test = np.array([0, 1, 0, 0])
predicted_test = np.array([1, 0, 0, 0])
auc = roc_auc_score(expected_test, predicted_test, average='weighted', multi_class="ovr")
print('ROC AUC: %f' % auc)
print(classification_report(expected, np.round(predicted), target_names=target_names))

Accuracy: 0.9927357891394093
Precision: 0.9928965462384829
Recall: 0.9927357891394093
f1-score: 0.9927852714648865
ROC AUC: 0.333333
              precision    recall  f1-score   support

         DoS       0.00      0.00      0.00         0
         U2R       1.00      1.00      1.00      9234
         R2L       1.00      0.55      0.71        11
       Probe       0.82      0.84      0.83       209
      Normal       0.98      0.98      0.98      2289
     Unknown       0.99      0.99      0.99     13449

   micro avg       0.99      0.99      0.99     25192
   macro avg       0.80      0.73      0.75     25192
weighted avg       0.99      0.99      0.99     25192
 samples avg       0.99      0.99      0.99     25192



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning:

Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning:

F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/loc